# New ML Active Learning Workflow
---

# Import Modules

In [ ]:
import os
import sys

sys.path.insert(0, os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/ml_convergence_plots"))

import pickle
import random
import copy

import pandas as pd
import numpy as np
import plotly.plotly as py

from ase.visualize import view
from ase import io

# #############################################################################
# #############################################################################
from catlearn.regression import GaussianProcess
import gpflow

# #############################################################################
# #############################################################################
from methods import process_data_for_plot, get_layout

# Script Inputs

# Read Data

In [ ]:
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/parsing_chris_dft_data",
    "df_dft_calcs.pickle")
with open(path_i, "rb") as fle:
    df_dft_calcs = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling",
    "df_oqmd_data.pickle")
with open(path_i, "rb") as fle:
    df_oqmd_data = pickle.load(fle)

df_train = pd.concat([df_oqmd_data, df_dft_calcs], sort=False)
# Removing missing data
df_train = df_train[df_train["atoms"].notnull()]
df_train = df_train.drop(["energy", "force_max", "force_sum"], axis=1)

# #############################################################################
# #############################################################################
# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "chris_prototypes_structures",
    "data_structures.pickle")
with open(path_i, "rb") as fle:
    df_struct = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "chris_prototypes_structures",
    "data_prototypes.pickle")
with open(path_i, "rb") as fle:
    df_proto = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "data/ml_irox_data",
    "unique_ids.csv")
df_ids = pd.read_csv(path_i)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/00_ml_workflow/190611_new_workflow",
    "data",
    "df_features_pca.pickle",
    )
with open(path_i, "rb") as fle:
    df_features_pca = pickle.load(fle)

# Finding Structure that is closest to beta-IrO3 phase

In [ ]:
# AB3_8_ik_glm2_66
# npvumg7g93

df_proto[df_proto["spacegroup_i"] == 66]

# Finding Structure that is closest to alpha-IrO3 phase

In [ ]:
# AB3_2_b_e_167
# 8k7expx2bp

df_proto[df_proto["spacegroup_i"] == 167]


row_i = df_struct.loc["8k7expx2bp"]
view(row_i["atoms"])

# view(df_struct.loc["8k7expx2bp"]["atoms"])

In [ ]:
row_i

# Getting minimum energy line trace in animation